In [3]:
import sys, os
sys.path.append(os.path.dirname(os.getcwd()))

In [32]:
import PyRandomUtils as pru
from pdb import pm
import utils.CardUtils as cu
import numpy as np
import matplotlib.pyplot as plt
import utils.sobol_seq as ss

In [6]:
batch_size = 36 #number of deals in a batch
num_batches = 1000

In [73]:
DECK = np.array(range(1,53), dtype=np.int8)
ts = cu.theoretical_probabilities
def shuffled(gen):
    tmp = np.array(DECK)
    pru.shuffle(gen, tmp)
    return tmp

In [40]:
class SobolGen(pru.PRNG):
    def __init__(self, seed):
        self.seed = seed
        
    def rand(self):
        r, self.seed = ss.i4_sobol(1, self.seed)
        return r

In [41]:
#bad prng
bad = pru.LCG(mod=2**5, a=5, seed=1)

#deterministic gen
sobol  = SobolGen(1)

#good prng
good = pru.PyRandGen(1)

In [120]:
mydeck = shuffled(bad)
mydeck

array([41, 29, 17,  5, 42, 30, 19,  9, 43, 32, 22, 13, 44, 34, 25,  3, 45,
       36, 28, 21, 46, 38, 31, 24, 20, 40, 14,  7, 47,  4, 37, 10, 48, 12,
       26, 18, 49, 16,  6, 35, 50,  2, 11, 27, 51, 23, 33, 39, 52,  8,  1,
       15], dtype=int8)

In [46]:
def make_ts(gen, batches=num_batches, batch_size=batch_size):
    ts = [sum((cu.get_features(shuffled(gen)) for i in range(batch_size))) / batch_size
           for j in range(num_batches)]
    return np.array(ts).T

In [48]:
%%time
ts_bad = make_ts(bad)

CPU times: user 14.6 s, sys: 155 ms, total: 14.8 s
Wall time: 14.2 s


In [49]:
%%time
ts_sobol = make_ts(sobol)

CPU times: user 55.8 s, sys: 227 ms, total: 56.1 s
Wall time: 55.5 s


In [50]:
%%time
ts_good = make_ts(good)

CPU times: user 16 s, sys: 255 ms, total: 16.2 s
Wall time: 15.6 s


In [130]:
def make_graphs(ts):
    dim = len(ts.shape[0])
    fig, ax = plt.subplots(dim,1, figsize = (15, 100))

    for i in range(dim):
        plt.subplot(dim,1,i+1)
        plt.title(cu.feature_string[i],fontsize=16)
        plt.plot(ts_sobol[i] - tp[i])

def print_means(ts):
    means = np.apply_along_axis(np.mean, 1, ts)
    tp = cu.theoretical_probabilities
    print("{:22}{:^22}{:^22}{:^22}".format("Feature", "p", "p_hat", "p-p_hat"))
    for i in range(len(cu.feature_string)):
        print("{:20} {: 20.18f} {: 20.18f} {: 20.18f}".format(cu.feature_string[i], 
                                           tp[i], 
                                           means[i], 
                                           tp[i]-means[i]))

In [132]:
print_means(ts_good)

Feature                         p                   p_hat                p-p_hat        
ls4                   0.350800000000000001  0.351763888888888931 -0.000963888888888931
ls5                   0.443400000000000016  0.442631944444444481  0.000768055555555536
ls6                   0.165500000000000008  0.165284722222222236  0.000215277777777773
ls7                   0.035299999999999998  0.035472222222222217 -0.000172222222222219
ls8                   0.004700000000000000  0.004541666666666667  0.000158333333333333
ls9                   0.000370000000000000  0.000277777777777778  0.000092222222222222
ls10                  0.000017000000000000  0.000027777777777778 -0.000010777777777778
ls11                  0.000000300000000000  0.000000000000000000  0.000000300000000000
ls12                  0.000000003000000000  0.000000000000000000  0.000000003000000000
ls13                  0.000000000006000000  0.000000000000000000  0.000000000006000000
ss0                   0.05120000000000000